In [3]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator
%matplotlib inline


Bad key text.latex.unicode in file /Users/caochenjie/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file /Users/caochenjie/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key nbagg.transparent in file /Users/caochenjie/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 426 ('nbagg.transparent: True')
You probably need to get an updated matplotlibr

In [4]:
anime = pd.read_csv('./archive/anime.csv')
rating = pd.read_csv('./archive/rating.csv')

In [5]:
rating.rating.replace({-1: np.nan}, inplace = True)
rating.head()

,user_id,anime_id,rating
0,1,20,NaN
1,1,24,NaN
2,1,79,NaN
3,1,226,NaN
4,1,241,NaN


In [6]:
anime['type'].unique()

array(['Movie', 'TV', 'OVA', 'Special', 'Music', 'ONA', nan], dtype=object)

In [7]:
anime_tv = anime[anime['type']=='TV']
print(anime_tv.shape)
anime_tv.head()

(3787, 7)


,anime_id,name,genre,type,episodes,rating,members
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
5,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10,9.15,93351


In [8]:
# Join the two dataframes on the anime_id columns
merged = rating.merge(anime_tv, left_on = 'anime_id', right_on = 'anime_id', suffixes= ['_user', ''])
merged.rename(columns = {'rating_user':'user_rating'}, inplace = True)

In [9]:
merged.shape

(5283596, 9)

In [10]:
# For computing reasons I'm limiting the dataframe length to 10,000 user
merged=merged[['user_id', 'name', 'user_rating']]
merged_sub= merged[merged.user_id <= 10000]
print(merged_sub.shape)
merged_sub.head()

(703813, 3)


,user_id,name,user_rating
0,1,Naruto,NaN
1,3,Naruto,8.0
2,5,Naruto,6.0
3,6,Naruto,NaN
4,10,Naruto,NaN


In [11]:
# 自动过滤掉全nan的行列
piv = merged_sub.pivot_table(index=['user_id'], columns=['name'], values='user_rating')

In [12]:
print(piv.shape)
piv.head()

(9387, 2708)


name,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,009-1,07-Ghost,11eyes,12-sai.: Chicchana Mune no Tokimeki,3 Choume no Tama: Uchi no Tama Shirimasenka?,30-sai no Hoken Taiiku,91 Days,...,"Zone of the Enders: Dolores, I",Zukkoke Knight: Don De La Mancha,ef: A Tale of Melodies.,ef: A Tale of Memories.,gdgd Fairies,gdgd Fairies 2,iDOLM@STER Xenoglossia,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Note: As we are subtracting the mean from each rating to standardize
# all users with only one rating or who had rated everything the same will be dropped

# Normalize the values
piv_norm = piv.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)


# Drop all columns containing only zeros representing users who did not rate
piv_norm.fillna(0, inplace=True)
piv_norm = piv_norm.T
piv_norm = piv_norm.loc[:, (piv_norm != 0).any(axis=0)]

In [14]:
print(piv_norm.shape)
piv_norm.head()

(2708, 8508)


user_id,3,5,7,8,10,11,12,14,16,17,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
name,,,,,,,,,,,,,,,,,,,,,
.hack//Roots,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
.hack//Sign,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
.hack//Tasogare no Udewa Densetsu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
009-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
07-Ghost,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.519231,0.0,0.0,0.0


In [15]:
# Our data needs to be in a sparse matrix format to be read by the following functions

piv_sparse = sp.sparse.csr_matrix(piv_norm.values)

In [16]:
# These matrices show us the computed cosine similarity values 
# between each user/user array pair and item/item array pair.
item_similarity = cosine_similarity(piv_sparse)
user_similarity = cosine_similarity(piv_sparse.T)

In [17]:
print(item_similarity.shape)
print(user_similarity.shape)

(2708, 2708)
(8508, 8508)


In [18]:
# Inserting the similarity matricies into dataframe objects

item_sim_df = pd.DataFrame(item_similarity, index = piv_norm.index, columns = piv_norm.index)
user_sim_df = pd.DataFrame(user_similarity, index = piv_norm.columns, columns = piv_norm.columns)

In [20]:
user_sim_df.shape

(8508, 8508)

In [19]:
user_sim_df.head()

user_id,3,5,7,8,10,11,12,14,16,17,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
user_id,,,,,,,,,,,,,,,,,,,,,
3,1.000000,0.063993,0.167627,0.020365,0.125977,4.523309e-02,0.108629,0.005399,0.077388,0.248132,...,-0.109099,-0.145602,-0.017873,0.043000,0.054199,0.027335,-0.015971,0.075051,0.041128,0.068404
5,0.063993,1.000000,0.072457,0.016326,0.031464,5.346814e-03,0.094016,0.106924,0.007067,0.130260,...,0.004349,-0.053570,0.044240,0.012790,0.103616,0.168159,-0.014871,0.075040,0.022199,0.008838
7,0.167627,0.072457,1.000000,-0.013548,0.020401,-1.515745e-02,0.061591,0.122852,0.048526,0.165473,...,0.000000,0.000000,0.000631,0.020430,0.068743,0.091902,0.019716,0.127440,0.050940,-0.013605
8,0.020365,0.016326,-0.013548,1.000000,-0.052705,1.892828e-02,0.002786,-0.055035,0.000000,0.001459,...,-0.228218,0.000000,0.019007,0.043206,0.035957,-0.036623,0.000000,-0.041096,-0.018445,0.106221
10,0.125977,0.031464,0.020401,-0.052705,1.000000,-1.665335e-16,0.008811,0.044415,0.000000,0.071511,...,0.000000,0.000000,0.000000,0.208672,-0.015703,0.040062,0.000000,0.259914,0.058329,-0.050271


In [60]:
# This function will return the top 10 shows with the highest cosine similarity value

def top_animes(anime_name):
    count = 1
    print('Similar shows to {} include:\n'.format(anime_name))
    for item in item_sim_df.sort_values(by = anime_name, ascending = False).index[1:11]:
        print('No. {}: {}'.format(count, item))
        count +=1  

# This function will return the top 5 users with the highest similarity value 

def top_users(user):
    if user not in piv_norm.columns:
        return('No data available on user {}'.format(user))
    
    print('Most Similar Users:\n')
    sim_values = user_sim_df.sort_values(by=user, ascending=False).loc[:,user].tolist()[1:11]
    sim_users = user_sim_df.sort_values(by=user, ascending=False).index[1:11]
    zipped = zip(sim_users, sim_values,)
    for user, sim in zipped:
        print('User #{0}, Similarity value: {1:.2f}'.format(user, sim)) 
        
# This function constructs a list of lists containing the highest rated shows per similar user
# and returns the name of the show along with the frequency it appears in the list

def similar_user_recs(user):
    
    if user not in piv_norm.columns:
        return('No data available on user {}'.format(user))
    
    sim_users = user_sim_df.sort_values(by=user, ascending=False).index[1:11]
    best = []
    most_common = {}
    
    for i in sim_users:
        max_score = piv_norm.loc[:, i].max()
        best.append(piv_norm[piv_norm.loc[:, i]==max_score].index.tolist())
    for i in range(len(best)):
        for j in best[i]:
            if j in most_common:
                most_common[j] += 1
            else:
                most_common[j] = 1
    sorted_list = sorted(most_common.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_list[:5]  


# This function calculates the weighted average of similar users
# to determine a potential rating for an input user and show

def predicted_rating(anime_name, user):
    sim_users = user_sim_df.sort_values(by=user, ascending=False).index[1:1000]
    user_values = user_sim_df.sort_values(by=user, ascending=False).loc[:,user].tolist()[1:1000]
    rating_list = []
    weight_list = []
    for j, i in enumerate(sim_users):
        rating = piv.loc[i, anime_name]
        similarity = user_values[j]
        if np.isnan(rating):
            continue
        elif not np.isnan(rating):
            rating_list.append(rating*similarity)
            weight_list.append(similarity)
    return sum(rating_list)/sum(weight_list)  

In [57]:
top_animes('Cowboy Bebop')

Similar shows to Cowboy Bebop include:

No. 1: Samurai Champloo
No. 2: Tengen Toppa Gurren Lagann
No. 3: Baccano!
No. 4: Ghost in the Shell: Stand Alone Complex
No. 5: Neon Genesis Evangelion
No. 6: Ghost in the Shell: Stand Alone Complex 2nd GIG
No. 7: Fullmetal Alchemist: Brotherhood
No. 8: Steins;Gate
No. 9: Mushishi
No. 10: Great Teacher Onizuka


In [58]:
top_users(3)

Most Similar Users:

User #2986, Similarity value: 0.37
User #2411, Similarity value: 0.36
User #3681, Similarity value: 0.36
User #656, Similarity value: 0.35
User #298, Similarity value: 0.34
User #3028, Similarity value: 0.34
User #8436, Similarity value: 0.33
User #2038, Similarity value: 0.33
User #2374, Similarity value: 0.33
User #4233, Similarity value: 0.33


In [61]:
similar_user_recs(3)

[('Boku dake ga Inai Machi', 4),
 ('Shingeki no Kyojin', 4),
 ('Steins;Gate', 4),
 ('Fullmetal Alchemist: Brotherhood', 4),
 ('Clannad: After Story', 3)]

In [62]:
predicted_rating('Cowboy Bebop', 3)

8.499797904818438

In [63]:
# Creates a list of every show watched by user 3

watched = piv.T[piv.loc[3,:]>0].index.tolist()

In [64]:
# Make a list of the squared errors between actual and predicted value

errors = []
for i in watched:
    actual=piv.loc[3, i]
    predicted = predicted_rating(i, 3)
    errors.append((actual-predicted)**2)

In [65]:
# This is the average squared error for user 3
np.mean(errors)

0.8848212679397977